# GloVe : SVD와 skip-gram 결합한 모델
- 단어 간의 의미적 유사성을 벡터 공간에서 보다 잘 반영한다.(SVD와 skip-gram 결합한 모델)
- 전역 통계 활용 : 단어 간의 동시 출현 확률을 사용하여 벡터를 학습한다.
- 동시 출현 행렬 생성 : 행렬의 각 항목 xij은 단어 i와 단어 j가 같은 문맥에서 등장한 횟수를 우선적으로 구해준다.
- 비용 함수 : 두 개의 단어, 동시 출현 비율 모델링하는 비용함수를 만들어서 벡터를 학습한다.
- 유사성 유지 : 단어 간의 유사성을 벡터 공간에서 유지하려고 한다.(유사한 단어들은 임베딩 공간에서 가깝게 위치하고 있다.)

In [1]:
import tensorflow as tf


KeyboardInterrupt



In [2]:
import requests
import zipfile
import os

url = "http://nlp.stanford.edu/data/glove.6B.zip"
zip_file = 'golve.6B.zip'

response = requests.get(url)
with open(zip_file, 'wb') as f:
    f.write(response.content)

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
os.remove(zip_file)

In [4]:
import numpy as np
import os

path_to_glove_file = 'glove.6B.100d.txt'

embeddings_index = {}
with open(path_to_glove_file, encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ') # 각 줄이 단어 벡터로 이루어져 있어, 단어와 벡트 값으로 분리
        embeddings_index[word] = coefs # 벡터 문자열을 Numpy 배열로 변환

print(f'단어 벡터 개수 : {len(embeddings_index)}')

단어 벡터 개수 : 400000


In [7]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size)

val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size=batch_size)

test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size=batch_size)

text_only_train_ds = train_ds.map(lambda x, y, : x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [8]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode = 'int',
    output_sequence_length=max_length,
)

text_vectorization.adapt(text_only_train_ds)

In [9]:
int_train_ds = train_ds.map(
    lambda x, y : (text_vectorization(x), y),
    num_parallel_calls=4)

int_val_ds = val_ds.map(
    lambda x, y : (text_vectorization(x), y),
    num_parallel_calls=4)

int_test_ds = test_ds.map(
    lambda x, y : (text_vectorization(x), y),
    num_parallel_calls=4)

In [12]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))
print(word_index)

{'': 0, '[UNK]': 1, 'the': 2, 'and': 3, 'a': 4, 'of': 5, 'to': 6, 'is': 7, 'in': 8, 'it': 9, 'i': 10, 'this': 11, 'that': 12, 'br': 13, 'was': 14, 'as': 15, 'for': 16, 'with': 17, 'movie': 18, 'but': 19, 'film': 20, 'on': 21, 'not': 22, 'you': 23, 'are': 24, 'his': 25, 'have': 26, 'be': 27, 'he': 28, 'one': 29, 'its': 30, 'at': 31, 'all': 32, 'by': 33, 'an': 34, 'they': 35, 'who': 36, 'from': 37, 'so': 38, 'like': 39, 'her': 40, 'just': 41, 'or': 42, 'about': 43, 'has': 44, 'if': 45, 'out': 46, 'some': 47, 'there': 48, 'what': 49, 'good': 50, 'when': 51, 'more': 52, 'very': 53, 'she': 54, 'even': 55, 'my': 56, 'up': 57, 'no': 58, 'would': 59, 'time': 60, 'which': 61, 'only': 62, 'really': 63, 'story': 64, 'see': 65, 'had': 66, 'were': 67, 'their': 68, 'can': 69, 'me': 70, 'than': 71, 'we': 72, 'much': 73, 'well': 74, 'been': 75, 'get': 76, 'will': 77, 'into': 78, 'do': 79, 'other': 80, 'great': 81, 'because': 82, 'also': 83, 'people': 84, 'bad': 85, 'how': 86, 'first': 87, 'most': 88, 

In [17]:
embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word) # 현재 단어의 벡터
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [18]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.038194  , -0.24487001,  0.72812003, ..., -0.1459    ,
         0.82779998,  0.27061999],
       ...,
       [-0.078638  ,  0.054143  , -0.099305  , ...,  0.25646001,
        -0.18741   , -0.44277999],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
from tensorflow.keras import layers, initializers

embedding_layer = layers.Embedding(
    input_dim=max_tokens, # 사전 단어 수
    output_dim=embedding_dim, # 단어 벡터의 차원 수 
    embeddings_initializer=initializers.Constant(embedding_matrix), # 임베딩 매트릭스 초기화 방법
    trainable=False, # 임베딩 레이어의 가중치를 학습할 것인지?
    mask_zero=True # 마스킹
)

In [ ]:
inputs = keras.Input(shape=(None, ), dtype='int64') # 입력층
embedded = embedding_layer(inputs) # 입력된 결과를 임베딩 처리
x = layers.Bidirectional(layers.LSTM(32))(embedded) # LSTM
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x) # 시그모이드로 이진분류

model = keras.Model(inputs, outputs) # 모델 구현(출력)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint('glove_embeddings_sequence_model.h5', save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
462/625 [=====================>........] - ETA: 2:59 - loss: 0.5845 - accuracy: 0.6896

In [ ]:
# 정확도 출력
model = keras.models.load_model('glove_embeddings_sequence_model.h5')
print(f'테스트 정확도 : {model.evaluate(int_test_ds)[1]:.3f}')